## PySpark - Recomendadores - Filtrado Colaborativo

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("recomendador").getOrCreate()

In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

### Alternating Least Squares - ALS

ALS se utiliza para resolver el problema de la factorización matricial en sistemas de recomendación.
El objetivo de la factorización matricial en este contexto es encontrar dos matrices, una de **usuarios** y otra de **ítems**, que cuando se multiplican entre sí, aproximan la matriz de valoraciones (o interacciones) entre usuarios e ítems.

El algoritmo ALS busca encontrar estas dos matrices de manera iterativa, y lo hace a través de dos pasos que se alternan en cada iteración:

- Fijar la matriz de usuarios y resolver para la matriz de ítems. En este paso, se utiliza la regresión lineal para ajustar una matriz de ítems que pueda explicar la matriz de valoraciones conocidas. Para hacerlo, se utiliza la matriz de usuarios fija para predecir la matriz de valoraciones que debería haber para cada ítem. Una vez que se tienen estas predicciones, se ajusta la matriz de ítems para que las predicciones se ajusten lo mejor posible a la matriz de valoraciones conocida.

- Fijar la matriz de ítems y resolver para la matriz de usuarios. En este paso, se utiliza el mismo proceso que en el paso anterior, pero ahora se fija la matriz de ítems y se ajusta la matriz de usuarios para que las predicciones de valoraciones se ajusten lo mejor posible a la matriz de valoraciones conocida.

Estos dos pasos se repiten iterativamente hasta que se alcanza un punto de convergencia, es decir, hasta que las matrices de usuarios e ítems que se obtienen en una iteración no son muy diferentes de las obtenidas en la iteración anterior.

Es importante destacar que el algoritmo ALS no es determinista, es decir, que puede arrojar resultados ligeramente diferentes en cada ejecución, ya que en cada iteración se utilizan valores aleatorios para inicializar las matrices de usuarios e ítems.


In [ ]:
df = spark.read.csv(path        = "/kaggle/input/pyspark-ml-recomendador/ratings.csv",
                    inferSchema = True,
                    header      = True)

In [ ]:
df.show(5, truncate = False)

In [ ]:
# Eliminamos esta columna porque no la necesitamos y ahorramos espacio.

df = df.drop("timestamp")

In [ ]:
df.count()

In [ ]:
# Cortamos el df porque no se puede ejecutar por el tamaño en Kaggle.

print(f'Tamaño original: {df.count()}')

df = df.filter(df["userId"] < 10000)

print(f'Tamaño nuevo: {df.filter(df["userId"] < 10000).count()}')

In [ ]:
df.show()

In [ ]:
df.describe().show()

In [ ]:
train, test = df.randomSplit([0.8, 0.2], seed = 42)

In [ ]:
# Eliminamos el df original para ahorrar espacio.

del df

In [ ]:
als = ALS(maxIter           = 10,
          regParam          = 0.01,
          userCol           = "userId",
          itemCol           = "movieId",
          ratingCol         = "rating",
          coldStartStrategy = "drop")

model  = als.fit(train)

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.show()

In [ ]:
evaluator = RegressionEvaluator(metricName    = "rmse",
                                labelCol      = "rating",
                                predictionCol = "prediction")

rmse = evaluator.evaluate(predictions)

print(f"RMSE: {rmse}")

In [ ]:
usuario_activo = test.filter(test["userId"] == 42).select(["movieId", "userId"])

usuario_activo.show()

In [ ]:
recommendations = model.transform(usuario_activo)

recommendations.orderBy("prediction", ascending = False).show()

In [ ]:
################################################################################################################################